## Imports

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import wandb
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score

## Prepare data

In [19]:
path = '995,000_rows.csv'
df = pd.read_csv(path)

real_types = ['political', 'fake', 'satire', 'reliable', 'conspiracy', 'bias', 'junksci']
df = df[df['type'].notna() & (df['type'].isin(real_types))]
reliable_types = ['reliable', 'political']

/var/folders/08/xjbs0s0j72s556nv4_bvymhh0000gn/T/ipykernel_8403/4215835436.py:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


### Part 2
## Split data

In [20]:
X = np.array(df['content'])
y = np.array([int(t in reliable_types) for t in df['type'] ])

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=27)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=27)

## Simple Baseline Classifiers

### Random guessing

In [44]:
uniform_classifier = DummyClassifier(strategy='uniform', random_state=0)
uniform_classifier.fit(X_train, y_train)
y_pred = uniform_classifier.predict(X_val)

# Calculate metrics
uniform_accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='weighted')

# Display the scores
print(f"Accuracy of Uniform Random classifier: {accuracy:.4f}")
print(f"F1 score of Uniform Random classifier: {f1:.4f}\n")

Accuracy of Uniform Random classifier: 0.5005
F1 score of Uniform Random classifier: 0.5010


Guessing most frequent label

In [45]:
classifier = DummyClassifier(strategy='most_frequent', random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_val)

# Calculate metrics
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='weighted')

# Display the scores
print(f"Accuracy of Most Frequent classifier: {accuracy:.4f}")
print(f"F1 score of Most Frequent classifier: {f1:.4f}\n")

Accuracy of Most Frequent classifier: 0.5335
F1 score of Most Frequent classifier: 0.3712
